In [1]:
import ds_utils
import pandas as pd
df = pd.read_csv('train.csv')

y = df['Survived']
df = df.drop(columns=['Survived'])

train_num = len(df)
df = pd.concat((df, pd.read_csv('test.csv')))

In [2]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df['Age'].unique()

array([22.  , 38.  , 26.  , 35.  ,   nan, 54.  ,  2.  , 27.  , 14.  ,
        4.  , 58.  , 20.  , 39.  , 55.  , 31.  , 34.  , 15.  , 28.  ,
        8.  , 19.  , 40.  , 66.  , 42.  , 21.  , 18.  ,  3.  ,  7.  ,
       49.  , 29.  , 65.  , 28.5 ,  5.  , 11.  , 45.  , 17.  , 32.  ,
       16.  , 25.  ,  0.83, 30.  , 33.  , 23.  , 24.  , 46.  , 59.  ,
       71.  , 37.  , 47.  , 14.5 , 70.5 , 32.5 , 12.  ,  9.  , 36.5 ,
       51.  , 55.5 , 40.5 , 44.  ,  1.  , 61.  , 56.  , 50.  , 36.  ,
       45.5 , 20.5 , 62.  , 41.  , 52.  , 63.  , 23.5 ,  0.92, 43.  ,
       60.  , 10.  , 64.  , 13.  , 48.  ,  0.75, 53.  , 57.  , 80.  ,
       70.  , 24.5 ,  6.  ,  0.67, 30.5 ,  0.42, 34.5 , 74.  , 22.5 ,
       18.5 , 67.  , 76.  , 26.5 , 60.5 , 11.5 ,  0.33,  0.17, 38.5 ])

In [4]:
df['Name'].head(10)

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
5                                     Moran, Mr. James
6                              McCarthy, Mr. Timothy J
7                       Palsson, Master. Gosta Leonard
8    Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)
9                  Nasser, Mrs. Nicholas (Adele Achem)
Name: Name, dtype: object

In [5]:
df['LastName'] = df['Name'].apply(lambda x: x.split(',')[0])

In [6]:
df['LastName'].head()

0       Braund
1      Cumings
2    Heikkinen
3     Futrelle
4        Allen
Name: LastName, dtype: object

In [7]:
df['TicketDigits'] = df['Ticket'].apply(lambda x: x.split()[1] if len(x.split()) > 1 else x)

In [8]:
df['Title'] = df['Name'].apply(lambda x: x.split(',')[1].split('.')[0])

In [9]:
df['Title'].head()

0       Mr
1      Mrs
2     Miss
3      Mrs
4       Mr
Name: Title, dtype: object

In [10]:
df['FirstName'] = df['Name'].apply(lambda x: x.split(',')[1].split('.')[1].split(' ')[1])

In [11]:
df['FirstName'].head()

0       Owen
1       John
2      Laina
3    Jacques
4    William
Name: FirstName, dtype: object

In [12]:
df = df.drop(columns=['Ticket', 'Name'])

In [13]:
ds_utils.get_nan_col_proportions(df)

[('Age', 0.20091673032849502),
 ('Fare', 0.0007639419404125286),
 ('Cabin', 0.774637127578304),
 ('Embarked', 0.0015278838808250573)]

In [14]:
df['Cabin'].head(10)

0     NaN
1     C85
2     NaN
3    C123
4     NaN
5     NaN
6     E46
7     NaN
8     NaN
9     NaN
Name: Cabin, dtype: object

In [15]:
import numpy as np
df['Cabin'] = df['Cabin'].fillna(' ') 
df['CabinLetter'] = df.apply(lambda row: ' ' if row['Cabin'] == ' ' else row['Cabin'][0], axis=1)

In [17]:
df = df.drop(columns=['Cabin'])

In [18]:
import numpy as np

df['NanAge'] = df.apply(lambda row: 1 if row['Age'] == np.nan else 0, axis=1)
df['NanFare'] = df.apply(lambda row: 1 if row['Fare'] == np.nan else 0, axis=1)
df['NanEmbarked'] = df.apply(lambda row: 1 if row['Embarked'] == np.nan else 0, axis=1)

df['CabinLetter'] = df['CabinLetter'].replace(to_replace=' ', value=np.nan)
df['NanCabinLetter'] = df.apply(lambda row: 1 if row['CabinLetter'] == np.nan else 0, axis=1)

df['Embarked'] = df.groupby('Pclass')['Embarked'].transform(lambda x: x.fillna(x.mode()[0]))
df['CabinLetter'] = df.groupby('Embarked')['CabinLetter'].transform(lambda x: x.fillna(x.mode()))

In [19]:
df['Age'] = df.groupby('Title')['Age'].transform(lambda x: x.fillna(x.median()))
df['Fare'] = df.groupby('Pclass')['Fare'].transform(lambda x: x.fillna(x.median()))
df['Embarked'] = df.groupby('Pclass')['Embarked'].transform(lambda x: x.fillna(x.mode()[0]))

df['Infant'] = df['Age'].apply(lambda x: 1 if x <= 1 else 0)
df['Toddler'] = df['Age'].apply(lambda x: 1 if x <= 5 else 0)
df['Child'] = df['Age'].apply(lambda x: 1 if x < 13 else 0)
df['YoungTeen'] = df['Age'].apply(lambda x: 1 if 13 <= x <= 16 else 0)
df['YoungAdult'] = df['Age'].apply(lambda x: 1 if 16 <= x <= 21 else 0)
df['Adult'] = df['Age'].apply(lambda x: 1 if 21 <= x <= 30 else 0)
df['MidAdult'] = df['Age'].apply(lambda x: 1 if 30 <= x <= 40 else 0)
df['MiddleAged'] = df['Age'].apply(lambda x: 1 if 40 <= x <= 50 else 0)
df['Old'] = df['Age'].apply(lambda x: 1 if 50 < x else 0)

In [20]:
df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,LastName,TicketDigits,...,NanCabinLetter,Infant,Toddler,Child,YoungTeen,YoungAdult,Adult,MidAdult,MiddleAged,Old
0,1,3,male,22.0,1,0,7.2500,S,Braund,21171,...,0,0,0,0,0,0,1,0,0,0
1,2,1,female,38.0,1,0,71.2833,C,Cumings,17599,...,0,0,0,0,0,0,0,1,0,0
2,3,3,female,26.0,0,0,7.9250,S,Heikkinen,3101282,...,0,0,0,0,0,0,1,0,0,0
3,4,1,female,35.0,1,0,53.1000,S,Futrelle,113803,...,0,0,0,0,0,0,0,1,0,0
4,5,3,male,35.0,0,0,8.0500,S,Allen,373450,...,0,0,0,0,0,0,0,1,0,0


In [21]:
df.loc[df['TicketDigits'].str[-1] == '.'].head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,LastName,TicketDigits,...,NanCabinLetter,Infant,Toddler,Child,YoungTeen,YoungAdult,Adult,MidAdult,MiddleAged,Old
115,116,3,male,21.0,0,0,7.925,S,Pekoniemi,2.,...,0,0,0,0,0,1,1,0,0,0
173,174,3,male,21.0,0,0,7.925,S,Sivola,2.,...,0,0,0,0,0,1,1,0,0,0
243,244,3,male,22.0,0,0,7.125,S,Maenpaa,2.,...,0,0,0,0,0,0,1,0,0,0
382,383,3,male,32.0,0,0,7.925,S,Tikkanen,2.,...,0,0,0,0,0,0,0,1,0,0
400,401,3,male,39.0,0,0,7.925,S,Niskanen,2.,...,0,0,0,0,0,0,0,1,0,0


In [22]:
df['TicketDigits'] = df['TicketDigits'].apply(lambda x: x if x[-1] != '.' else x[:-1])

In [23]:
df.loc[df['TicketDigits'].str.len() < 5].head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,LastName,TicketDigits,...,NanCabinLetter,Infant,Toddler,Child,YoungTeen,YoungAdult,Adult,MidAdult,MiddleAged,Old
10,11,3,female,4.0,1,1,16.7000,S,Sandstrom,9549,...,0,0,1,1,0,0,0,0,0,0
12,13,3,male,20.0,0,0,8.0500,S,Saundercock,2151,...,0,0,0,0,0,1,0,0,0,0
19,20,3,female,35.5,0,0,7.2250,C,Masselmani,2649,...,0,0,0,0,0,0,0,1,0,0
26,27,3,male,29.0,0,0,7.2250,C,Emir,2631,...,0,0,0,0,0,0,1,0,0,0
36,37,3,male,29.0,0,0,7.2292,C,Mamee,2677,...,0,0,0,0,0,0,1,0,0,0


In [24]:
df['TicketDigits'] = df['TicketDigits'].apply(lambda x: x if x[-1] != '.' else x[:-1])

In [25]:
df.loc[df['TicketDigits'].str[-1] == '.']

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,LastName,TicketDigits,...,NanCabinLetter,Infant,Toddler,Child,YoungTeen,YoungAdult,Adult,MidAdult,MiddleAged,Old


In [26]:
df.loc[df['TicketDigits'] == 'LINE']

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,LastName,TicketDigits,...,NanCabinLetter,Infant,Toddler,Child,YoungTeen,YoungAdult,Adult,MidAdult,MiddleAged,Old
179,180,3,male,36.0,0,0,0.0,S,Leonard,LINE,...,0,0,0,0,0,0,0,1,0,0
271,272,3,male,25.0,0,0,0.0,S,Tornquist,LINE,...,0,0,0,0,0,0,1,0,0,0
302,303,3,male,19.0,0,0,0.0,S,Johnson,LINE,...,0,0,0,0,0,1,0,0,0,0
597,598,3,male,49.0,0,0,0.0,S,Johnson,LINE,...,0,0,0,0,0,0,0,0,1,0


In [27]:
df['TicketDigits'] = df['TicketDigits'].apply(lambda x: 1 if x == 'LINE' else x)
df['TicketDigits'] = df['TicketDigits'].apply(lambda x: 0 if x == 'Basle' else x)

In [28]:
df.loc[df['TicketDigits'] == 'LINE']

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,LastName,TicketDigits,...,NanCabinLetter,Infant,Toddler,Child,YoungTeen,YoungAdult,Adult,MidAdult,MiddleAged,Old


In [29]:
df['Title'].unique()

array([' Mr', ' Mrs', ' Miss', ' Master', ' Don', ' Rev', ' Dr', ' Mme',
       ' Ms', ' Major', ' Lady', ' Sir', ' Mlle', ' Col', ' Capt',
       ' the Countess', ' Jonkheer', ' Dona'], dtype=object)

In [30]:
df['Title'] = df['Title'].apply(lambda x: x.strip())
df['Title'] = df['Title'].apply(lambda x: x if x != 'Miss' else 'Ms')

In [31]:
df['Title'].unique()

array(['Mr', 'Mrs', 'Ms', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Major',
       'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess', 'Jonkheer',
       'Dona'], dtype=object)

In [32]:
df['Embarked'] = df['Embarked'].astype('category')
df['LastName'] = df['LastName'].astype('category')
df['FirstName'] = df['FirstName'].astype('category')
df['Title'] = df['Title'].astype('category')
df['TicketDigits'] = df['TicketDigits'].astype(int)
df['CabinLetter'] = df['CabinLetter'].astype('category')
df['Sex'] = df['Sex'].astype('category')
df['FamilySize'] = df['SibSp'] + df['Parch']
df['Alone'] = df['FamilySize'].apply(lambda x: 1 if x == 0 else 0)

In [33]:
df['French'] = df['Title'].apply(lambda x: 1 if x in ['Mlle', 'Mme'] else 0)
df['Spanish'] = df['Title'].apply(lambda x: 1 if x in ['Don', 'Dona'] else 0)
df['Military'] = df['Title'].apply(lambda x: 1 if x in ['Capt', 'Col', 'Major'] else 0)
df['Nobility'] = df['Title'].apply(lambda x: 1 if x in ['Don', 'Sir', 'the Countess', 'Lady'] else 0)

In [34]:
df[df['Title'] == 'Master'].head(20)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,LastName,TicketDigits,...,Adult,MidAdult,MiddleAged,Old,FamilySize,Alone,French,Spanish,Military,Nobility
7,8,3,male,2.00,3,1,21.0750,S,Palsson,349909,...,0,0,0,0,4,0,0,0,0,0
16,17,3,male,2.00,4,1,29.1250,Q,Rice,382652,...,0,0,0,0,5,0,0,0,0,0
50,51,3,male,7.00,4,1,39.6875,S,Panula,3101295,...,0,0,0,0,5,0,0,0,0,0
59,60,3,male,11.00,5,2,46.9000,S,Goodwin,2144,...,0,0,0,0,7,0,0,0,0,0
63,64,3,male,4.00,3,2,27.9000,S,Skoog,347088,...,0,0,0,0,5,0,0,0,0,0
65,66,3,male,4.00,1,1,15.2458,C,Moubarek,2661,...,0,0,0,0,2,0,0,0,0,0
78,79,2,male,0.83,0,2,29.0000,S,Caldwell,248738,...,0,0,0,0,2,0,0,0,0,0
125,126,3,male,12.00,1,0,11.2417,C,Nicola-Yarred,2651,...,0,0,0,0,1,0,0,0,0,0
159,160,3,male,4.00,8,2,69.5500,S,Sage,2343,...,0,0,0,0,10,0,0,0,0,0
164,165,3,male,1.00,4,1,39.6875,S,Panula,3101295,...,0,0,0,0,5,0,0,0,0,0


In [35]:
df['FirstName'] = df['FirstName'].apply(lambda x: x[1:] if x[0] == '(' else x)

In [36]:
df2 = df.copy()

In [37]:
df.dtypes

PassengerId          int64
Pclass               int64
Sex               category
Age                float64
SibSp                int64
Parch                int64
Fare               float64
Embarked          category
LastName          category
TicketDigits         int64
Title             category
FirstName           object
CabinLetter       category
NanAge               int64
NanFare              int64
NanEmbarked          int64
NanCabinLetter       int64
Infant               int64
Toddler              int64
Child                int64
YoungTeen            int64
YoungAdult           int64
Adult                int64
MidAdult             int64
MiddleAged           int64
Old                  int64
FamilySize           int64
Alone                int64
French               int64
Spanish              int64
Military             int64
Nobility             int64
dtype: object

In [38]:
df = pd.get_dummies(df)

In [39]:
df.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,TicketDigits,NanAge,NanFare,NanEmbarked,...,FirstName_Yousseff,FirstName_of,CabinLetter_A,CabinLetter_B,CabinLetter_C,CabinLetter_D,CabinLetter_E,CabinLetter_F,CabinLetter_G,CabinLetter_T
0,1,3,22.0,1,0,7.2500,21171,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,2,1,38.0,1,0,71.2833,17599,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,3,3,26.0,0,0,7.9250,3101282,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,35.0,1,0,53.1000,113803,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,5,3,35.0,0,0,8.0500,373450,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
df_train = df[:train_num].copy()
df_train['Survived'] = y
df_test = df[train_num:].copy()

In [41]:
df_train.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,TicketDigits,NanAge,NanFare,NanEmbarked,...,FirstName_of,CabinLetter_A,CabinLetter_B,CabinLetter_C,CabinLetter_D,CabinLetter_E,CabinLetter_F,CabinLetter_G,CabinLetter_T,Survived
0,1,3,22.0,1,0,7.2500,21171,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,2,1,38.0,1,0,71.2833,17599,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,3,3,26.0,0,0,7.9250,3101282,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,1,35.0,1,0,53.1000,113803,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,5,3,35.0,0,0,8.0500,373450,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
df_train.to_csv('clean_train.csv', index=False)
df_test.to_csv('clean_test.csv', index=False)